In [1]:
# Program to pickle historical data for SD of nse underlyings

# STATUS: Incomplete (See To-do)

#***          Start ib_insync (run once)       *****
#_______________________________________________

from ib_insync import *
util.startLoop()
# ib=IB().connect('127.0.0.1', 7496, clientId=3) # kavi TWS live
ib=IB().connect('127.0.0.1', 4001, clientId=3) # kavi IBG live

In [95]:
import pandas as pd
import numpy as np
import sys
import datetime

#*****            Initializations           ****
#_______________________________________________

load_from_disk = True   # For underlying symbol list, ohlc

#******         Paths and variables         ****
#_______________________________________________

datapath = r'./zdata/'
market = 'NSE'

#******   Error catch in list comprehension  ****
#________________________________________________

def catch(func, handle=lambda e : e, *args, **kwargs):
    '''List comprehension error catcher'''
    try:
        return func(*args, **kwargs)
    except Exception as e:
        return np.nan

#*****    Get the symbols list    ****
#_____________________________________

# Load symbols from disk if available
if load_from_disk:
    try:
        symbols = list(pd.read_csv('./zdata/nse_eq_symbols.csv'))
    except FileNotFoundError as e:
        print('csv file not found')
        sys.exit(0)

# Rectified for IBKR
isymbols = [i[0:9] for i in symbols]
isymbols = [s.replace('&','') for s in isymbols]

In [ ]:
#*****    Append OHLC for missing dates   ****
#_____________________________________________

# Get pickled data from disk
df_raw = pd.read_pickle(datapath+r'df_ohlc.pkl')

# Simulate missing dates
df_raw = df_raw[df_raw.D < datetime.date(2018,10,1)]  # DELETE in production

# Get the last date of the pickle 
last_dt = df_raw.loc[(df_raw.Symbol == df_raw.Symbol[0]), 'D'].max()

# Get the latest date of ib.reqHistoricalData
c = Stock(symbol=df_raw.Symbol[0], exchange=market) # first symbol's contract

latest_dt = ib.reqHistoricalData(contract=c,
                                 endDateTime= '', durationStr='1 D', barSizeSetting='1 day', 
                                 whatToShow='Trades', useRTH=True)[0].date

delta_days = latest_dt - last_dt    # additonal days required

#.....      Correct the duration string    ....
#..............................................
duration = delta_days.days

if delta_days.days > 0:
    
    # check data available for delta days
    duration = str(delta_days.days)+' D'
    dates_list = [d.date for d in ib.reqHistoricalData(contract=c, endDateTime='',
                               durationStr=duration, barSizeSetting='1 day', 
                               whatToShow='Trades', useRTH=True)]
    
    # Correct delta days duration to points of data available.
    # This would be the duration string for ib.reqHistoricalData delta
    duration = str(len([d for d in dates_list if d >= last_dt])) + ' D'
    
    #.....  Make dataframe of delta dates    ....
    #............................................
    
    existing_symbols = df_raw.Symbol.unique()
    
    contracts = [Stock(symbol=s, exchange=market) for s in existing_symbols]
    
    add_hist_dict = dict([(c.symbol, ib.reqHistoricalData(contract=c, endDateTime='', 
                                    durationStr=duration, barSizeSetting='1 day', 
                                    whatToShow='Trades', useRTH=True)) 
                    for c in contracts])
    
    # make the delta dataframe
    data = [(k, i.date, i.open, i.high, i.low, i.close, i.average, i.volume) 
                  for k, v in add_hist_dict.items() 
                  for i in v]
    df_date_delta = pd.DataFrame(data, columns=['Symbol', 'D', 'O', 'H', 'L', 'C', 'Avg', 'Vol'])


# To-do
* Check on M&M and L&TFIN scrips. Why are they missing?
* Add history for missing equity symbols
* Add history for missing index symbols
* Clean-up 'First-Run' for generating base history pickle

In [61]:
# Append to create the new data frame
df_new = pd.concat(objs=[df_raw, df_date_delta], 
                   axis=0).sort_values(['Symbol', 'D']).reset_index(drop=True)

In [70]:
#***            FIRST-RUN OHLC              ***
#______________________________________________


# qualify contracts
contracts = [Stock(symbol=s, exchange=market) for s in isymbols]
qual_stocks = ib.qualifyContracts(*contracts)

# make a historical dictionary
hist_dict = dict([(c.symbol, ib.reqHistoricalData(contract=c, endDateTime='', durationStr='365 D', 
                                    barSizeSetting='1 day',  whatToShow='Trades', useRTH=True)) 
                  for c in contracts])

# make a dataframe
data = [(k, i.date, i.open, i.high, i.low, i.close, i.average, i.volume) 
              for k, v in hist_dict.items() 
              for i in v]

df = pd.DataFrame(data, columns=['Symbol', 'D', 'O', 'H', 'L', 'C', 'Avg', 'Vol'])

# pickle the dataframe
df.to_pickle(datapath+r'df_ohlc.pkl')

In [83]:
len(df[df.Symbol == 'MM'])

0

In [ ]:
df.groupby('D')

In [88]:
a = df.Symbol.unique()

In [92]:
sorted(a)

['ACC',
 'ADANIENT',
 'ADANIPORT',
 'ADANIPOWE',
 'AJANTPHAR',
 'ALBK',
 'AMARAJABA',
 'AMBUJACEM',
 'APOLLOHOS',
 'APOLLOTYR',
 'ARVIND',
 'ASHOKLEY',
 'ASIANPAIN',
 'AUROPHARM',
 'AXISBANK',
 'BAJAJ-AUT',
 'BAJAJFINS',
 'BAJFINANC',
 'BALKRISIN',
 'BANKBAROD',
 'BANKINDIA',
 'BATAINDIA',
 'BEL',
 'BEML',
 'BERGEPAIN',
 'BHARATFIN',
 'BHARATFOR',
 'BHARTIART',
 'BHEL',
 'BIOCON',
 'BOSCHLTD',
 'BPCL',
 'BRITANNIA',
 'CADILAHC',
 'CANBK',
 'CANFINHOM',
 'CAPF',
 'CASTROLIN',
 'CEATLTD',
 'CENTURYTE',
 'CESC',
 'CGPOWER',
 'CHENNPETR',
 'CHOLAFIN',
 'CIPLA',
 'COALINDIA',
 'COLPAL',
 'CONCOR',
 'CUMMINSIN',
 'DABUR',
 'DALMIABHA',
 'DCBBANK',
 'DHFL',
 'DISHTV',
 'DIVISLAB',
 'DLF',
 'DRREDDY',
 'EICHERMOT',
 'ENGINERSI',
 'EQUITAS',
 'ESCORTS',
 'EXIDEIND',
 'FEDERALBN',
 'GAIL',
 'GLENMARK',
 'GMRINFRA',
 'GODFRYPHL',
 'GODREJCP',
 'GODREJIND',
 'GRANULES',
 'GRASIM',
 'GSFC',
 'HAVELLS',
 'HCC',
 'HCLTECH',
 'HDFC',
 'HDFCBANK',
 'HEROMOTOC',
 'HEXAWARE',
 'HINDALCO',
 'HINDPETRO',
 

In [94]:
a = pd.read_csv(datapath+'_raw_equity.csv')
sorted(a.Symbol)

['ACC',
 'ADANIENT',
 'ADANIPORTS',
 'ADANIPOWER',
 'AJANTPHARM',
 'ALBK',
 'AMARAJABAT',
 'AMBUJACEM',
 'APOLLOHOSP',
 'APOLLOTYRE',
 'ARVIND',
 'ASHOKLEY',
 'ASIANPAINT',
 'AUROPHARMA',
 'AXISBANK',
 'BAJAJ-AUTO',
 'BAJAJFINSV',
 'BAJFINANCE',
 'BALKRISIND',
 'BANKBARODA',
 'BANKINDIA',
 'BATAINDIA',
 'BEL',
 'BEML',
 'BERGEPAINT',
 'BHARATFIN',
 'BHARATFORG',
 'BHARTIARTL',
 'BHEL',
 'BIOCON',
 'BOSCHLTD',
 'BPCL',
 'BRITANNIA',
 'CADILAHC',
 'CANBK',
 'CANFINHOME',
 'CAPF',
 'CASTROLIND',
 'CEATLTD',
 'CENTURYTEX',
 'CESC',
 'CGPOWER',
 'CHENNPETRO',
 'CHOLAFIN',
 'CIPLA',
 'COALINDIA',
 'COLPAL',
 'CONCOR',
 'CUMMINSIND',
 'DABUR',
 'DALMIABHA',
 'DCBBANK',
 'DHFL',
 'DISHTV',
 'DIVISLAB',
 'DLF',
 'DRREDDY',
 'EICHERMOT',
 'ENGINERSIN',
 'EQUITAS',
 'ESCORTS',
 'EXIDEIND',
 'FEDERALBNK',
 'GAIL',
 'GLENMARK',
 'GMRINFRA',
 'GODFRYPHLP',
 'GODREJCP',
 'GODREJIND',
 'GRANULES',
 'GRASIM',
 'GSFC',
 'HAVELLS',
 'HCC',
 'HCLTECH',
 'HDFC',
 'HDFCBANK',
 'HEROMOTOCO',
 'HEXAWARE',
 'H

In [97]:
sorted(isymbols)

['ACC',
 'ADANIENT',
 'ADANIPORT',
 'ADANIPOWE',
 'AJANTPHAR',
 'ALBK',
 'AMARAJABA',
 'AMBUJACEM',
 'APOLLOHOS',
 'APOLLOTYR',
 'ARVIND',
 'ASHOKLEY',
 'ASIANPAIN',
 'AUROPHARM',
 'AXISBANK',
 'BAJAJ-AUT',
 'BAJAJFINS',
 'BAJFINANC',
 'BALKRISIN',
 'BANKBAROD',
 'BANKINDIA',
 'BATAINDIA',
 'BEL',
 'BEML',
 'BERGEPAIN',
 'BHARATFIN',
 'BHARATFOR',
 'BHARTIART',
 'BHEL',
 'BIOCON',
 'BOSCHLTD',
 'BPCL',
 'BRITANNIA',
 'CADILAHC',
 'CANBK',
 'CANFINHOM',
 'CAPF',
 'CASTROLIN',
 'CEATLTD',
 'CENTURYTE',
 'CESC',
 'CGPOWER',
 'CHENNPETR',
 'CHOLAFIN',
 'CIPLA',
 'COALINDIA',
 'COLPAL',
 'CONCOR',
 'CUMMINSIN',
 'DABUR',
 'DALMIABHA',
 'DCBBANK',
 'DHFL',
 'DISHTV',
 'DIVISLAB',
 'DLF',
 'DRREDDY',
 'EICHERMOT',
 'ENGINERSI',
 'EQUITAS',
 'ESCORTS',
 'EXIDEIND',
 'FEDERALBN',
 'GAIL',
 'GLENMARK',
 'GMRINFRA',
 'GODFRYPHL',
 'GODREJCP',
 'GODREJIND',
 'GRANULES',
 'GRASIM',
 'GSFC',
 'HAVELLS',
 'HCC',
 'HCLTECH',
 'HDFC',
 'HDFCBANK',
 'HEROMOTOC',
 'HEXAWARE',
 'HINDALCO',
 'HINDPETRO',
 